# Chapter 9: Working with Range Data

## A Crash Course in Genomic Ranges and Coordinate Systems

* **Definition of a Range:** A range is an **integer interval** that specifies a **consecutive subsequence of positions** on a single biological sequence (like a chromosome).
* **Discrete Positions:** Ranges use integers because base pairs are discrete; there are no fractional genomic positions.
* **Genomic Region Definition:** To fully specify a unique **genomic region or position**, **three** necessary pieces of information are needed:
    1.  **Chromosome Name (Sequence Name):** Identifies the specific sequence (e.g., chromosome, scaffold, or contig) the range is on. This naming scheme is **not standardized** (e.g., "chr17," "22," "MT," or "scaffold\_1648") and is always dependent on a specific **genome assembly version**.
    2.  **Range:** Specifies the subsequence using a **start position** and an **end position**.
    3.  **Strand:** Specifies whether the feature resides on the **forward (positive)** or **reverse (negative)** strand of the double-stranded DNA. This is necessary because many features, such as **protein-coding exons**, are **strand-specific** to make biological sense when translated.

### Reference Genome Versions

* **Genomes are Dynamic:** Reference genomes are continuously **changing and improving**, leading to changes in the coordinate system over time.
* **Coordinates are Version-Specific:** A specific genomic range (e.g., $chr15: \text{position}$) **does not refer to the same physical location** across different genome versions (e.g., GRCh38 vs. hg19).
* **Reproducibility Requirement:** It is **vital to specify the exact reference genome version** used (e.g., *human GRCh38*, *Zea mays AGPv3*) and for all collaborators to use the same version for data comparison.
* **Need for Remapping (Liftover):** Converting genomic data from one version's coordinate system to another's is necessary but tedious.
* **Remapping Tools:** Established tools exist for this "liftover" task:
    * **[CrossMap](http://crossmap.sourceforge.net/):** A command-line tool for converting various data formats (BED, VCF, etc.) between versions.
    * **[Comparative Genome Viewer (CGV)]():** A web-based tool which displays assembly-assembly whole genome alignments to help you quickly compare eukaryotic genome assemblies and easily identify genomic changes that may be significant to biology and evolution.
    * **LiftOver:** A web-based tool hosted by the UCSC Genome Browser.

### Coordinate range systems
There are two different flavors of range systems used by bioinformatics data formats:
* 0-based coordinate system, with half-closed, half-open intervals ($[start, end)$).
* 1-based coordinate system, with closed intervals ($[start, end]$).

For example, R uses 1-based indexing for its vectors and strings, and extracting a portion of a
string with `substr()` uses closed intervals. Here is the list bioinformatics tools with their range systems:

| Format/library               | Type      |
| :--------------------------- | :-------- |
| BED                          | 0-based   |
| GTF                          | 1-based   |
| GFF                          | 1-based   |
| SAM                          | 1-based   |
| BAM                          | 0-based   |
| VCF                          | 1-based   |
| BCF                          | 0-based   |
| Wiggle                       | 1-based   |
| GenomicRanges                | 1-based   |
| BLAST                        | 1-based   |
| GenBank/EMBL Feature Table   | 1-based   |

#### Comparison between 0-based and 1-based coordinate systems

* **Calculating Range Width:**
    * **0-based system:** Width is calculated simply and intuitively as **$end - start$**.
    * **1-based system:** Width requires the less intuitive calculation: **$end - start + 1$**.
* **Zero-Width Features (0-based advantage):**
    * The **0-based system naturally supports zero-width features** (e.g., $[12, 12)$).
    * This is useful for representing **positions *between* bases**, such as a restriction enzyme cut site.
    * The **1-based system** requires the smallest feature to be 1 base wide, sometimes using awkward notations (e.g., $[30, 29]$) to represent zero-width points.

## An Interactive Introduction to Range Data with GenomicRanges

### Installing and Working with Bioconductor Packages

**Bioconductor** is an **open-source software project** that creates and serves as a **repository for bioinformatics packages written in R**, emphasizing tools for **high-throughput data**.

* **Release Schedule:** New Bioconductor packages are released on a **strict, set schedule twice a year**.
* **Version Coordination:** Each Bioconductor release is tightly **coordinated with a specific version of R**.
* **Motivation for Coordination:** This strict system ensures that all packages are **thoroughly tested** before public release.
* **Development Access:** Users who require the very latest features can access a package's **development branch**.

we’ll touch on some of Bioconductor’s core packages:

* **GenomicRanges:** Used to represent and work with genomic ranges.
* **GenomicFeatures:** Used to represent and work with ranges that represent gene models and other features of a genome (genes, exons, UTRs, transcripts, etc.).
* **Biostrings and BSgenome:** Used for manipulating genomic sequence data in R (we’ll cover the subset of these packages used for extracting sequences from ranges).
* **rtracklayer:** Used for reading in common bioinformatics formats like BED, GTF/GFF, and WIG.

Installing packages from **Bioconductor** is done using a different method than installing standard CRAN packages.

Here's the step-by-step process:

#### Step 1: Install the BiocManager Package

First, you need to install the **`BiocManager`** package, which is the recommended installer for all Bioconductor packages. This package is hosted on CRAN, so you install it the standard way:

```r
install.packages("BiocManager") 
```

#### Step 2: Install Bioconductor Packages

Once `BiocManager` is installed, you use its main function, **`BiocManager::install()`**, to install any Bioconductor package.

For example, to install the `GenomicRanges` package, you run:

```r
BiocManager::install("GenomicRanges") 
```

This function automatically handles dependencies, ensures you're installing the correct package versions for your current R version, and manages the appropriate repositories.

#### Installing Multiple Packages

You can install multiple packages in a single call by providing a character vector:

```r
BiocManager::install(c("DESeq2", "edgeR", "TCRsampler"))
```

#### Installing from Specific Repositories

Bioconductor maintains three repositories:

  * **Software (Default)**: For tools like `DESeq2`.
  * **Annotation**: For annotation data packages like `org.Hs.eg.db`.
  * **Experiment**: For experimental data packages.

The `BiocManager::install()` function checks all necessary repositories by default, so you typically don't need to specify anything extra.

### Useful functions for package management

#### CRAN packages

* **Find package paths**: If you want to find the exact location of a specific package that has been loaded or is installed, you can use the `find.package()` function:

```r
# Example: Find the path for the 'ggplot2' package
find.package("ggplot2") 
```

* **List of old packages:** The `old.packages()` function checks the versions of the packages currently installed in your library against the versions available on CRAN.

* **Updating Outdated CRAN Packages:** Once you know which packages are outdated, you can use the `update.packages()` function to bring them all up to date.

#### Bioconductor packages

* **Checking Package Validity:** The `BiocManager::valid()` function checks your entire R environment against the currently recommended Bioconductor release.

```r
BiocManager::valid()
```

* **If everything is up to date**, the function will return `TRUE`.
* **If packages are outdated or inconsistent**, it will print a detailed report listing the packages that are either:
  * **Out of date** (a newer version is available).
  * **Too new** (installed from a different source and ahead of the recommended Bioconductor version).
  * **Missing** (required packages that aren't installed).

* **Automatically Updating Outdated Packages:** If `BiocManager::valid()` returns a list of problematic packages, you can usually update them all at once by running the command with no arguments:

```r
BiocManager::install()
```

When run this way, `BiocManager` checks for outdated packages and attempts to bring all of them, along with their dependencies, up to the version compatible with your current Bioconductor release. You'll be prompted to confirm which packages you want to update.

We run `remove.packages("package_name", lib = system_lib_path)` to remove a packaged installed on `system_lib_path` directory.

### Storing Generic Ranges with IRanges

The ranges we create with the IRanges package are called IRanges objects. We can create ranges with the `IRanges()` function:

```r
> rng <- IRanges(start=4, end=13)
> rng
# IRanges of length 1
#     start end width
# [1]     4  13    10
```

The `IRanges()` constructor (a function that creates a new object) can take vector arguments, creating an IRanges object containing many ranges:

```r
> x <- IRanges(start=c(4, 7, 2, 20), end=c(13, 7, 5, 23))
> x
IRanges of length 4
    start end width
[1]     4  13    10
[2]     7   7     1
[3]     2   5     4
[4]    20  23     4
```

* `names()`: To set the names of ranges.
* `str()`: To view classes slots (methods and attributes). We use accessor functions to get parts of an IRanges object. For example, you can access the start positions, end positions, and widths of each range in this object with the methods `start()`, `end()`, and `width()`.
* These functions also work with `<-` to set start, end, and width position. For example, we could increment a range’s end position by 4 positions with:

```r
> end(x) <- end(x) + 4
> x
IRanges of length 4
      start end width
[1] a     4  17    14
[2] b     7  11     5
[3] c     2   5     4
[4] d    20  23     4
```

* `range()`: Returns the span of the ranges kept in an IRanges object.
* We can subset IRanges just as we would any other R objects (vectors, dataframes, matrices), using either numeric, logical, or character (name) index.

```r
> x[2:4]
> x[start(x) > 2]
> x[width(x) > 5]
> x['a'] 
> x[c('a', 'd')] 
```

* `c()`: To combine two or more IRanges objects:

```r
> a <- IRanges(start=7, width=4)
> b <- IRanges(start=2, end=5)
> c(a, b)
IRanges of length 2
    start end width
[1]     7  10     4
[2]     2   5     4
```

### Basic Range Operations: Arithmetic, Transformations, and Set Operations

* **Beyond Data Frames**: `IRanges` objects are a **specialized class** for storing generic range data (start and end points), offering capabilities far beyond a standard data frame.
* **Specialized Methods**: The core advantage is the inclusion of **specialized methods** in the `IRanges` package, which simplify complex genomics data analysis tasks.
* **Method Consistency**: These same methods are implemented in the `GenomicRanges` package, allowing them to work similarly on **`GRanges` objects** (which add genomic context like chromosome and strand) as they do on generic `IRanges` objects.
* **Range Arithmetic**: `IRanges` objects support **arithmetic operations** (`+`, `-`, `*`) to easily modify range size:
    * **Addition (`+`) and Subtraction (`-`)**: **Grows or shrinks** a range **symmetrically** (e.g., adding a buffer region upstream and downstream).
    * **Multiplication (`*`)**: Supported, but division (`/`) is not, as it makes no sense for ranges.

* `restrict()`: Cuts a set of ranges such that they fall inside of a certain bound:

```r
> y <- IRanges(start=c(4, 6, 10, 12), width=13)
> y
IRanges of length 4
    start end width
[1]     4  16    13
[2]     6  18    13
[3]     10 22    13
[4]     12 24    13
> restrict(y, 5, 10)
IRanges of length 3
    start end width
[1]     5  10     6
[2]     6  10     5
[3]    10 10      1
```

* `flank()`: Returns the regions that flank (are on the side of) each range in an IRanges object. `flank()` is useful in creating ranges upstream and downstream of protein coding genes that could contain promoter sequences.
* `reduce()`: Takes a set of possibly overlapping ranges and reduces them to a set of nonoverlapping ranges that cover the same positions.
* `gaps()`: Returns the gaps (uncovered portions) between ranges. It has numerous applications in genomics: creating intron ranges between exons ranges, finding gaps in coverage, defining intragenic
regions between genic regions, and more. If you’d like `gaps()` to include these gaps, specify the start and end positions in gaps (e.g., `gaps(alns, start=1, end=60)`).


#### Sets operations for IRanges objects

* **Ranges as Sets of Integers**: An `IRanges` object represents a set of consecutive integers (e.g., `IRanges(start=4, end=7)` is the set $\{4, 5, 6, 7\}$).
* **Analogous Set Operations**: This allows for powerful range manipulations that mimic standard set theory:
    * **Difference**: $\text{setdiff}()$
    * **Intersection**: $\text{intersect}()$
    * **Union**: $\text{union}()$
    * **Complement/Gaps**: $\text{gaps}()$ (Finds regions *not* covered by the ranges).
* **Utility**: These operations simplify numerous data analysis tasks, particularly in genomics, by providing a fast and intuitive way to compare and combine genomic intervals.

#### Standard vs Pairwise Set Operation

* **Standard Set Operations (Global)**: Set operations (like `setdiff()`, `intersect()`, `union()`) treat an `IRanges` object as if its ranges have been **collapsed** first using the `reduce()` function. This means that overlapping ranges are considered to be part of the same underlying set of integers for the calculation.
* **Pairwise Set Operations (Local)**: A separate group of functions performs set operations **range-wise** (pairwise):
    * These functions require **two `IRanges` objects of equal length**.
    * They operate on the ranges at the **same index** in each object (e.g., the first range in object A with the first range in object B, the second with the second, and so on).
    * The pairwise functions are: **`psetdiff()`**, **`pintersect()`**, **`punion()`**, and **`pgap()`**.

### Finding Overlapping Ranges

* **Overlaps are Essential:** Finding overlaps is a fundamental requirement for numerous genomics analysis tasks.
* **Connecting Data to Features:** Overlaps are used to connect experimental data (like aligned reads, variants, or peaks) to annotated biological features (like gene regions, methylation sites, or conserved regions).
* **Quantification and Isoforms:** In tasks like RNA-seq, overlaps are used to quantify cellular activity (expression) and identify different transcript isoforms.
* **Computational Efficiency:** Computing overlaps is a computationally intensive task (potentially involving billions of ranges), which is why it is essential to use existing libraries that implement advanced data structures and algorithms for efficiency.
* **Technical Importance:** It is vital to understand the different types of overlaps and select the appropriate one, as the technical details can have a drastic impact on the final biological result.

`findOverlaps(query, subject)`: Returns an object with class Hits, which stores overlaps:

```r
> qry <- IRanges(start=c(1, 26, 19, 11, 21, 7), end=c(16, 30, 19, 15, 24, 8),
names=letters[1:6])
> sbj <- IRanges(start=c(1, 19, 10), end=c(5, 29, 16), names=letters[24:26])
> hts <- findOverlaps(qry, sbj)
```

Thinking abstractly, overlaps represent a mapping between query and subject. Depending on how we find overlaps, each query can have many hits in different subjects.

![findOverlaps](images/findOverlaps.png)

* **Hits Object Structure:** The `Hits` object returned by `findOverlaps()` has two columns representing **indices**: one for the **query ranges** and one for the **subject ranges**.
* **Data Representation:** Each row in the `Hits` object pairs the index of a query range that overlaps with the index of the subject range it overlaps.
* **Accessor Functions:** These indices can be extracted using two specific functions:
    * `queryHits()`: Retrieves the indices for the overlapping query ranges.
    * `subjectHits()`: Retrieves the indices for the overlapping subject ranges.
* **Practical Use:** The indices are typically used to subset or retrieve metadata (like **names**) from the original query and subject range objects:

```r
> names(qry)[queryHits(hts)]
[1] "a" "a" "b" "c" "d" "e"
> names(sbj)[subjectHits(hts)]
[1] "x" "z" "y" "y" "z" "y"
```
#### Some useful arguments of `findOverlaps()`

##### 'type'

Depending on our biological task, `type="any"` may not be the best form of overlap. For example, we could limit our overlap results to only include query ranges that fall entirely within subject ranges with `type="within"`.

```r
hts_within <- findOverlaps(qry, sbj, type="within") 
```

##### `select`
The `select` parameter determines how `findOverlaps()` handles one-to-many overlaps (where one query range hits multiple subject ranges).

* **Default Behavior**:
    * **`select = "all"` (Default)**: Returns **all** overlapping hits for every query range, typically resulting in a specialized data structure (like an `hts` object) where the number of rows is greater than the number of query ranges.
* **One-Hit-Per-Query Options**: When `select` is set to return only one overlap per query range, the result is a simple **integer vector**, where each element corresponds to a query range in the order it appeared in the query set.
    * **`select = "first"`**: Returns the **first** overlapping subject range found.
    * **`select = "last"`**: Returns the **last** overlapping subject range found.
    * **`select = "arbitrary"`**: Returns **one** of the overlapping subject ranges arbitrarily.
    * If a query range has no overlap, the corresponding element in the resulting vector is **`NA`**.

#### A few notes

* **Computational Challenge**: Naively checking overlaps between $Q$ query ranges and $S$ subject ranges requires a massive $Q \times S$ comparisons, making the operation computationally expensive.
* **Exploiting Order**: Overlap computation can be optimized because ranges are ordered along a sequence. If a query range ends at position $X$, it cannot possibly overlap any subject range that starts beyond $X$.
* **The Solution: Interval Tree**: The **interval tree** is a specialized data structure that exploits this ordering property. It allows the system to efficiently skip unnecessary comparisons.
* **Best Use Case**: Interval trees are most effective when the task involves finding overlaps of **many query ranges** against a **fixed set of subject ranges**.
* **Efficiency Gains**: The interval tree is built once using the subject ranges. Although building the tree takes some initial time, it can then be reused repeatedly for fast overlap searching with all the query ranges, yielding significant performance gains overall.
* **Built-in Solution**: The `IRanges` package provides the **`NCList` class**.
* **Automated Use**: This class automatically utilizes interval trees **under the hood** to enable fast overlap finding.
* **Simple Creation**: Creating an `NCList` object from an existing `IRanges` object is straightforward:

```r
> sbj_nc <- NCList(sbj)
> sbj_nc
NCList object with 3 ranges and 0 metadata columns:
        start       end     width
    <integer> <integer> <integer>
  x         1         5         5
  y        19        29        11
  z        10        16         7
> class(sbj_nc)
[1] "NCList"
attr(,"package")
[1] "IRanges"
```

Several key functions and properties for manipulating and analyzing the results of the `findOverlaps()` operation, which are stored in a **`Hits` object**.

* **`Hits` Coercion**: A `Hits` object can be converted into a standard matrix using the**`as.matrix()`** function.
* **Counting Query Overlaps**:
    * **`countQueryHits()`**: Returns a vector indicating **how many subject ranges** each individual query range overlaps.
    * **Labeling**: The `setNames()` function is used to apply the original query range names to the resulting count vector for clarity.
* **Counting Subject Overlaps**:
    * **`countSubjectHits()`**: Returns a vector indicating **how many query ranges** overlap each individual subject range.
    * **Labeling**: `setNames()` is similarly used to label these counts with the subject range names.
* **Extracting Overlapping Regions**:
    * The **`ranges()`** function can be called on the `Hits` object (along with the original query and subject ranges) to extract the exact coordinates of the **intersecting regions**.
* **Difference from `intersect()`**: When used with a `Hits` object, `ranges()` differs from the `intersect()` set operation:
    * **`intersect()`** performs a global operation and **reduces** overlapping regions into the minimal set of unique intersecting ranges.
    * **`ranges()`** (with a `Hits` object) returns **separate intersecting ranges** for every specific hit pair, preserving the relationship found by `findOverlaps()`.

```r
> countQueryHits(hts)
[1] 2 1 1 1 1 0

> setNames(object = countQueryHits(hts), names(qry))
a b c d e f 
2 1 1 1 1 0 
> countSubjectHits(hts)
[1] 1 3 2
> setNames(object = countSubjectHits(hts), names(sbj))
x y z 
1 3 2 
```
